# Junção dos dados de Cliente com a base do IBGE

Este notebook tem como objetivo realizar a junção de duas bases de dados, base de Clientes CAIXA e a base de Área de ponderação do IBGE.

Serão utilizadas as bases:
 - Amostra de Minas Gerais dos Clientes CAIXA - **cli_end_amostra_minas_gerais_validos.csv**;
 - Amostra de endereços de Minas Gerais da Empresa Brasileira de Correios e Telégrafos - **ect_amostra_minas_gerais.csv**;
 - Base de CEP por Setores Censitários - **cadastro_nacional_enderecos_fins_estatisticos.csv**
 - Base de Área de Ponderação por Setores Censitários - **Áreas de Ponderao por Setores Censitários.csv**
 
 ---
 Importação da biblioteca Pandas para manipulação de dados de forma tabular.

In [1]:
import time
start = time.time() # inicia o tempo de execução

import pandas as pd
import re
pd.set_option('float_format', '{:f}'.format) # Visualização de números decimais com uma casa.

Leitura do primeiro arquivo do conjunto de dados de Clientes CAIXA.

In [2]:
df = pd.read_csv("../dados/notebook/faixa_renda_estimada/faixa_renda_estimada_limpeza.csv", delimiter=';')

In [3]:
df['valida'] = (df['nu_cep_full'] / 1000000).astype(int)

In [4]:
df_valido = df[df['valida'] != 0]
df_invalido = df[df['valida'] == 0]

In [5]:
del df

In [6]:
print(df_valido.shape, df_invalido.shape)

(5246793, 12) (1532839, 12)


In [7]:
df_valido

,nu_pessoa_p17,nu_cep_full,nu_pessoa,idade,ic_tipo_renda,ic_sexo,nu_estdo_civil_p27,de_estado_civil,vr_renda_bruta,vr_renda_liquida,dt_apuracao_renda,valida
0,8480186189848,37550000,0,0,?,?,0,0,0.000000,0.000000,?,37
1,9840064467984,31230460,0,0,?,?,0,0,0.000000,0.000000,?,31
2,6910060431691,35500200,0,0,?,?,0,0,0.000000,0.000000,?,35
3,5590187325559,39670000,0,0,?,?,0,0,0.000000,0.000000,?,39
4,5600102483560,39390000,0,0,?,?,0,0,0.000000,0.000000,?,39
...,...,...,...,...,...,...,...,...,...,...,...,...
5435040,470143547047,38500000,470143547047,21,I,F,1,7,0.000000,0.000000,2015-07-27,38
5435041,3380222118338,36406204,3380222118338,50,I,M,1,7,0.000000,1200.000000,2018-10-16,36
5435042,8620063421862,32215000,0,0,?,?,0,0,0.000000,0.000000,?,32
5435043,6730051464673,33140820,6730051464673,51,?,M,0,0,0.000000,0.000000,?,33


Criação de uma variável chamada "**corte**" para a realização de pequenas quebras em partes devido a quantidade de recurso computacional.

In [8]:
particao = int(input("Digite a quantidade de partições que se deseja: "))

Digite a quantidade de partições que se deseja: 100


In [9]:
corte = int((len(df_valido) / particao))

Criação de um método de divisão do conjunto de dados em 'n' partes.

In [10]:
df_new = []
contador = 0
multiplicador = 2
corte_base = corte
for i in range(particao):
    #print(contador, corte, multiplicador)
    df_new.append(df_valido[contador:corte])
    contador = corte
    corte = corte_base * multiplicador
    multiplicador += 1

Deleta o conjunto de dados principal.

In [11]:
colunas = df_valido.columns

In [12]:
colunas

Index(['nu_pessoa_p17', 'nu_cep_full', 'nu_pessoa', 'idade', 'ic_tipo_renda',
       'ic_sexo', 'nu_estdo_civil_p27', 'de_estado_civil', 'vr_renda_bruta',
       'vr_renda_liquida', 'dt_apuracao_renda', 'valida'],
      dtype='object')

In [13]:
del df_valido

Leitura do segundo arquivo de CEPs por Setores Censitários.

In [14]:
ibge_end = pd.read_csv("../dados/dominio/cadastro_nacional_enderecos_fins_estatisticos.csv", delimiter = ';')

Comando Linux para a criação de uma pasta temporária para a manipulação dos dados divididos.

In [15]:
!if [ -d "../dados/tmp" ]; then echo "Pasta está limpa!" && rm -rf ../dados/tmp/* ; else echo "Pasta criada com sucesso!" && mkdir -p ../dados/tmp; fi

Pasta está limpa!


Interação dos dez (10) conjuntos de dados realizando uma junção (**LEFT JOIN**) entre os arquivos de Clientes CAIXA e o arquivo de CEPs por Setores Censitários.

In [16]:
id_count = 1
for i in df_new:
    #print(i.shape)
    merged_end = pd.merge (i, ibge_end, how='left', left_on='nu_cep_full', right_on='CEP')
    #print(merged_end.shape)
    merged_end['rank_setor'] = merged_end.groupby(['nu_pessoa_p17', 'nu_cep_full', 'nu_pessoa'])['Código setor censitário'].rank(method='first')
    #print(merged_end.shape)
    merged_end = merged_end[merged_end['rank_setor'] == 1]
    #print(merged_end.shape)
    merged_end.to_csv('../dados/tmp/df_new' + str(id_count) + '.csv', index=False, encoding='utf-8', sep=';')
    del merged_end
    id_count += 1
    #print('\n\n')

Comando Linux gerando um arquivo com todos os nomes dos arquivos .csv que foram gerados no processo anterior.

In [17]:
!ls -ltr ../dados/tmp/*.csv | cut -d\  -f9 > ../dados/tmp/list_csv.txt

Comando Linux realizando a limpeza das linhas em branco do arquivo.

In [18]:
!sed -i '/^$/d' ../dados/tmp/list_csv.txt

Interação no arquivo **list_csv.txt** fazendo a leitura dos dez (10) arquivos e gerando somente um (1).

In [19]:
!while read a ; do cat $a >> ../dados/tmp/df_new.csv ; done < ../dados/tmp/list_csv.txt

Com a junção dos dez (10) arquivos em somente um (1), foi utilizado o comando Linux para substituir todos os cabeçalhos que vieram para o arquivo mantendo o primero, primeira linha de cabeçalho.

In [20]:
!sed -i "1! s/nu_pessoa_p17;nu_cep_full;nu_pessoa;idade;ic_tipo_renda;ic_sexo;nu_estdo_civil_p27;de_estado_civil;vr_renda_bruta;vr_renda_liquida;dt_apuracao_renda;valida;Código setor censitário;CEP;rank_setor//g" ../dados/tmp/df_new.csv

Com a eliminação das linhas de cabeçalhos, foi executado o comando Linux para a eliminação das linhas em branco.

In [21]:
!sed -i '/^$/d' ../dados/tmp/df_new.csv

Deleção dos DataFrames para a liberação de memória.

In [22]:
del ibge_end

Comando Linux para a remorção de todos os dez (10) arquivos gerados deixando somente o principal, este contendo todo o conteúdo dos dez (10).

In [23]:
!rm -rf ../dados/tmp/df_new?*.csv

---
Segunda junção de dados, importação dos dados da primeira junção com os dados de endereço da Empresa Brasileira de Correios e Telégrafos.

In [24]:
merged_end = pd.read_csv("../dados/tmp/df_new.csv", delimiter = ';')

In [25]:
merged_end.shape

(2628470, 15)

In [26]:
cep = pd.read_csv("../dados/amostra_correios/ect_amostra_minas_gerais.csv", delimiter = ';')
#cep.head()

In [27]:
merged_end.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2628470 entries, 0 to 2628469
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   nu_pessoa_p17            int64  
 1   nu_cep_full              int64  
 2   nu_pessoa                int64  
 3   idade                    int64  
 4   ic_tipo_renda            object 
 5   ic_sexo                  object 
 6   nu_estdo_civil_p27       int64  
 7   de_estado_civil          int64  
 8   vr_renda_bruta           float64
 9   vr_renda_liquida         float64
 10  dt_apuracao_renda        object 
 11  valida                   int64  
 12  Código setor censitário  float64
 13  CEP                      float64
 14  rank_setor               float64
dtypes: float64(5), int64(7), object(3)
memory usage: 300.8+ MB


Realização da segunda junção dos dados.

In [28]:
merged2 = pd.merge (merged_end,cep, how='left', left_on='CEP', right_on='int_cep')

Deleção dos DataFrames para a liberação de memória.

In [29]:
del merged_end
del cep

Criação do ranqueamento pelo "**nu_pessoa**" e "**nu_cep**" no atributo "**Código setor censitário**".

In [30]:
merged2['rank_setor'] = merged2.groupby(['nu_pessoa_p17', 'nu_cep_full'])['Código setor censitário'].rank(method='first')

Filtro do ranqueamento dos Clientes e CEPs para evitar duplicidade.

In [31]:
merged2 = merged2[merged2['rank_setor']==1]

Leitura do conjunto de dados de Áreas de Ponderação por Setores Censitários "**Áreas de Ponderao por Setores Censitários.csv**".

In [32]:
area_setor = pd.read_csv ('../dados/dominio/Áreas de Ponderao por Setores Censitários.csv', delimiter=';')

Junção dos conjuntos de Clientes CAIXA por Setores Censitários pelo conjunto de Setores Censitários por Área de Ponderação. Esta junção é para encontrar a Área de Ponderação com o dado do Setor Censitário.

In [33]:
merged2 = pd.merge(merged2,area_setor, how='left', left_on='Código setor censitário', right_on='Setor')

Remorção das colunas duplicadas ou de ranqueamento dos dados.

In [34]:
merged2 = merged2.drop(columns=['rank_setor','valida','cep','CEP','int_cep','Código setor censitário'])

In [35]:
merged3 = pd.concat([merged2,df_invalido], ignore_index=True)

Inclusão de zero (**0**) para valores nulos e alteração do tipo dos dados para **inteiro**.

In [36]:
merged3['Área de ponderação'] = merged3['Área de ponderação'].fillna(0)
merged3['Setor'] = merged3['Setor'].fillna(0)
merged3['mun_nu'] = merged3['mun_nu'].fillna(0)
merged3['Área de ponderação'] = merged3['Área de ponderação'].astype(int)
merged3['Setor'] = merged3['Setor'].astype(int)
merged3['mun_nu'] = merged3['mun_nu'].astype(int)

In [37]:
merged3.isna().sum()

nu_pessoa_p17               0
nu_cep_full                 0
nu_pessoa                   0
idade                       0
ic_tipo_renda               0
ic_sexo                     0
nu_estdo_civil_p27          0
de_estado_civil             0
vr_renda_bruta              0
vr_renda_liquida            0
dt_apuracao_renda           0
log_nu                2918566
ufe_sg                2918566
loc_nu                2918566
bai_nu_ini            2918566
bai_nu_fim            2918566
log_no                2918566
log_complemento       3940561
tlo_tx                2918566
log_sta_tlo           2918566
log_no_abrev          2918566
bai_no                2918566
bai_no_abrev          2918566
mun_nu                      0
Área de ponderação          0
Setor                       0
valida                2628426
dtype: int64

In [38]:
import numpy as np

merged3[merged3.select_dtypes(exclude=[np.number]).columns] = \
merged3[merged3.select_dtypes(exclude=[np.number]).columns].fillna('?')

In [39]:
merged3.isna().sum()

nu_pessoa_p17               0
nu_cep_full                 0
nu_pessoa                   0
idade                       0
ic_tipo_renda               0
ic_sexo                     0
nu_estdo_civil_p27          0
de_estado_civil             0
vr_renda_bruta              0
vr_renda_liquida            0
dt_apuracao_renda           0
log_nu                2918566
ufe_sg                      0
loc_nu                2918566
bai_nu_ini            2918566
bai_nu_fim            2918566
log_no                      0
log_complemento             0
tlo_tx                      0
log_sta_tlo                 0
log_no_abrev                0
bai_no                      0
bai_no_abrev                0
mun_nu                      0
Área de ponderação          0
Setor                       0
valida                2628426
dtype: int64

In [40]:
merged3[merged3.select_dtypes(include=['float64']).columns] = \
merged3[merged3.select_dtypes(include=['float64']).columns].fillna(0)

In [41]:
merged3.isna().sum()

nu_pessoa_p17         0
nu_cep_full           0
nu_pessoa             0
idade                 0
ic_tipo_renda         0
ic_sexo               0
nu_estdo_civil_p27    0
de_estado_civil       0
vr_renda_bruta        0
vr_renda_liquida      0
dt_apuracao_renda     0
log_nu                0
ufe_sg                0
loc_nu                0
bai_nu_ini            0
bai_nu_fim            0
log_no                0
log_complemento       0
tlo_tx                0
log_sta_tlo           0
log_no_abrev          0
bai_no                0
bai_no_abrev          0
mun_nu                0
Área de ponderação    0
Setor                 0
valida                0
dtype: int64

In [42]:
merged3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4161265 entries, 0 to 4161264
Data columns (total 27 columns):
 #   Column              Dtype  
---  ------              -----  
 0   nu_pessoa_p17       int64  
 1   nu_cep_full         int64  
 2   nu_pessoa           int64  
 3   idade               int64  
 4   ic_tipo_renda       object 
 5   ic_sexo             object 
 6   nu_estdo_civil_p27  int64  
 7   de_estado_civil     int64  
 8   vr_renda_bruta      float64
 9   vr_renda_liquida    float64
 10  dt_apuracao_renda   object 
 11  log_nu              float64
 12  ufe_sg              object 
 13  loc_nu              float64
 14  bai_nu_ini          float64
 15  bai_nu_fim          float64
 16  log_no              object 
 17  log_complemento     object 
 18  tlo_tx              object 
 19  log_sta_tlo         object 
 20  log_no_abrev        object 
 21  bai_no              object 
 22  bai_no_abrev        object 
 23  mun_nu              int64  
 24  Área de ponderação  int6

Exportação dos dados no formato "**csv**".

In [43]:
merged3.to_csv('../dados/notebook/cliente_area_de_ponderacao/cliente_area_de_ponderacao.csv', index=False, encoding='utf-8', sep=';')

Remorção dos arquivos temporários.

In [44]:
!rm -rf ../dados/tmp

In [45]:
elapsed = time.time() - start # informa o tempo de execução

In [46]:
elapsed / 60

5.633825620015462